In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Phase 1

In [18]:

BASE_DIR = Path("../") 
RAW_DATA_DIR = BASE_DIR / "data" / "raw"
PROCESSED_DATA_DIR = BASE_DIR / "processed" / "raw"
CUSTOMERS_CSV = RAW_DATA_DIR / "customers.csv"
PRODUCTS_CSV = RAW_DATA_DIR / "products.csv"
SALES_CSV = RAW_DATA_DIR / "sales_transactions.csv"

In [19]:
class Repository:
    def __init__(self, raw_data_dir, processed_data_dir):
        self.raw_data_dir = raw_data_dir
        self.processed_data_dir = processed_data_dir
        
    def load_customers(self):
        return pd.read_csv(self.raw_data_dir / "customers.csv")

    def load_products(self):
        return pd.read_csv(self.raw_data_dir / "products.csv")

    def load_sales(self):
        return pd.read_csv(self.raw_data_dir / "sales_transactions.csv")
    
    def save_customers(self, df):
        df.to_csv(self.processed_data_dir / "customers.csv", index=False)

    def save_products(self, df):
        df.to_csv(self.processed_data_dir / "products.csv", index=False)

    def save_sales(self, df):
        df.to_csv(self.processed_data_dir / "sales_transactions.csv", index=False)

In [20]:
repo=Repository(RAW_DATA_DIR,PROCESSED_DATA_DIR)
customer=repo.load_customers()
product=repo.load_products()
transactions=repo.load_sales()

In [21]:
customer.head()


,CustomerID,Name,Region,Signup_Date
0,CUST-001,Customer_1,east,2020-01-05
1,CUST-002,Customer_2,East,2020-01-12
2,CUST-003,Customer_3,West,2020-01-19
3,CUST-004,Customer_4,midwest,2020-01-26
4,CUST-005,Customer_5,midwest,2020-02-02


In [22]:
customer.describe()

,CustomerID,Name,Region,Signup_Date
count,105,105,105,105
unique,100,100,10,100
top,CUST-008,Customer_8,South,2020-02-23
freq,2,2,23,2


In [23]:
customer.info()

<class 'pandas.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   CustomerID   105 non-null    str  
 1   Name         105 non-null    str  
 2   Region       105 non-null    str  
 3   Signup_Date  105 non-null    str  
dtypes: str(4)
memory usage: 3.4 KB


In [24]:
# sign up date should be DateTime Not String
customer['Signup_Date']=pd.to_datetime(customer['Signup_Date'], errors='coerce')

In [25]:
customer.info()

<class 'pandas.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   CustomerID   105 non-null    str           
 1   Name         105 non-null    str           
 2   Region       105 non-null    str           
 3   Signup_Date  105 non-null    datetime64[us]
dtypes: datetime64[us](1), str(3)
memory usage: 3.4 KB


In [26]:
product.describe()

,Price
count,20.000000
mean,283.633000
std,121.485899
min,63.690000
25%,205.180000
50%,303.745000
75%,362.327500
max,464.980000


In [27]:
product.info()

<class 'pandas.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ProductID  20 non-null     str    
 1   Category   20 non-null     str    
 2   Price      20 non-null     float64
dtypes: float64(1), str(2)
memory usage: 612.0 bytes


In [28]:
transactions.describe()

,TransactionID,Quantity,Discount
count,1000.000000,1000.000000,1000.000000
mean,10500.500000,9.882000,0.085800
std,288.819436,70.281801,0.072998
min,10001.000000,-9.000000,0.000000
25%,10250.750000,3.000000,0.050000
50%,10500.500000,5.000000,0.050000
75%,10750.250000,7.000000,0.100000
max,11000.000000,1000.000000,0.200000


In [29]:
transactions.info()

<class 'pandas.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TransactionID  1000 non-null   int64  
 1   CustomerID     1000 non-null   str    
 2   ProductID      1000 non-null   str    
 3   Date           995 non-null    str    
 4   Quantity       1000 non-null   int64  
 5   Discount       1000 non-null   float64
 6   Total_Amount   971 non-null    str    
dtypes: float64(1), int64(2), str(4)
memory usage: 54.8 KB


# Problems with Data (Transactions)
- (Incorrect Data Types )
    - Total amount is str
    - Transaction Date is str
- There are multible leading and trailing spaces
- missing values in Date and total amount 
    - 5 in Dates
    - 29 in Total_Amount
- 42 Check_Receipt in Total_Amount
- 10 Invalid_Date exists
- Some Total_Amount got EUR and others doesn't
- Quantity values
    - Check for negative quantities (verify with df[df['Quantity'] < 0])
- there are outliers in the Quantity column

### Data Cleaning

In [30]:
# clean the product table (remove leading and trailing spaces)
product['Category'] = product['Category'].str.strip().str.title()

# clean the customer table (remove leading and trailing spaces)
customer['Region'] = customer['Region'].str.strip().str.title()
customer['Name'] = customer['Name'].str.strip().str.title()

# convert the Signup_Date into DateTime (done in a previous sell)
customer['Signup_Date'] = pd.to_datetime(customer['Signup_Date'], errors='coerce')

### handling missing values for Date ?

In [31]:
transactions['Quantity'].skew()
# since > 1, then it's skewed
# since there is outlier or skewness in the dataset we will fill the missing values with median not mean

np.float64(14.021126545817783)

In [32]:
# Converting the Date column in transactions to DateTime
transactions['Date']=pd.to_datetime(transactions['Date'], errors='coerce')

# Remove the EUR 
transactions['Total_Amount'] = (
    transactions['Total_Amount']
    .astype(str)          
    .str.replace('EUR', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.strip()
)

# Converting the Total_Amount column in transactions to Numeric
transactions['Total_Amount'] = pd.to_numeric(transactions['Total_Amount'], errors='coerce')

# remove from the Total_Amount column the Check_Receipt values
transactions = transactions[transactions['Total_Amount'].notna()]

In [33]:
#Remove negative Quantity
transactions = transactions[transactions['Quantity'] >= 0]

In [34]:

# Drop rows where Date is NaT (including "Invalid_Date")
transactions = transactions.dropna(subset=['Date'])

In [35]:
transactions.info()

<class 'pandas.DataFrame'>
Index: 734 entries, 3 to 999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   TransactionID  734 non-null    int64         
 1   CustomerID     734 non-null    str           
 2   ProductID      734 non-null    str           
 3   Date           734 non-null    datetime64[us]
 4   Quantity       734 non-null    int64         
 5   Discount       734 non-null    float64       
 6   Total_Amount   734 non-null    float64       
dtypes: datetime64[us](1), float64(2), int64(2), str(2)
memory usage: 45.9 KB


In [ ]:
# Phase 2

In [ ]:
# Merge sales with customers
sales_customers = transactions.merge(customer, on='CustomerID', how='left')

# Merge the result with products
full_data = sales_customers.merge(product, on='ProductID', how='left')

print(full_data.head())
print(full_data.info())

   TransactionID CustomerID ProductID                Date  Quantity  Discount  \
0          10004   CUST-008  PROD-006 2023-01-01 03:00:00         1      0.00   
1          10004   CUST-008  PROD-006 2023-01-01 03:00:00         1      0.00   
2          10006   CUST-045  PROD-004 2023-01-01 05:00:00         5      0.05   
3          10007   CUST-078  PROD-018 2023-01-01 06:00:00         1      0.10   
4          10008   CUST-017  PROD-017 2023-01-01 07:00:00         3      0.10   

   Total_Amount         Name Region Signup_Date   Category   Price  
0        148.00   Customer_8   West  2020-02-23  Furniture  148.00  
1        148.00   Customer_8   West  2020-02-23  Furniture  148.00  
2        856.62  Customer_45  South  2020-11-08  Furniture  180.34  
3         57.32  Customer_78   West  2021-06-27   Clothing   63.69  
4       1088.83  Customer_17  South  2020-04-26  Furniture  403.27  
<class 'pandas.DataFrame'>
RangeIndex: 773 entries, 0 to 772
Data columns (total 12 columns):
 #   

In [ ]:
# Phase 3

In [38]:
full_data[['Quantity', 'Price', 'Discount', 'Total_Amount', 'Calculated_Revenue']].head(10)

KeyError: "['Calculated_Revenue'] not in index"

In [ ]:
#Feature Engineering

# calculated revenue
full_data['Calculated_Revenue'] = full_data['Quantity'] * full_data['Price'] * (1 - full_data['Discount'])

#compare with the total amount (original dirty )
full_data['Revenue_Difference'] = full_data['Total_Amount'] - full_data['Calculated_Revenue']
print(full_data[['Total_Amount','Calculated_Revenue', 'Revenue_Difference']].head(10))

   Total_Amount  Calculated_Revenue  Revenue_Difference
0        403.17            362.9430             40.2270
1        403.17            362.9430             40.2270
2       2822.54           2681.7455            140.7945
3       2822.54           2681.7455            140.7945
4        148.00            148.0000              0.0000
5        148.00            148.0000              0.0000
6        856.62            856.6150              0.0050
7         57.32             57.3210             -0.0010
8       1088.83           1088.8290              0.0010
9        577.09            577.0880              0.0020


In [ ]:
#fill total amount with the calculated amount
full_data['Total_Amount'] = full_data['Total_Amount'].fillna(full_data['Calculated_Revenue'])

# Show full data after filling Total_Amount
full_data[['CustomerID','ProductID','Quantity','Price','Discount','Total_Amount','Calculated_Revenue']].head(20)